In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

Intitializing Scala interpreter ...

Spark Web UI available at http://36de47ca6b01:4042
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1595855995906)
SparkSession available as 'spark'


import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 3.0.0


In [5]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath: String = /home/jovyan/data/events-stream
modelPath: String = /home/jovyan/models/spark-ml-model
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(tweet,StringType,true))
inputDF: org.apache.spark.sql.DataFrame = [tweet: string]


6. После построения модели и ее сохранения на диск, необходимо запустить стриминг из ноутбука (tweet_feeder).
7. Далее необходимо применить модель в streaming mode. Заготовка для имплементации находится в ноутбуке (apply_model). В базовом подходе достаточно просто выводить результат применения модели. Обратите внимание что в качестве результата мы хотим видеть в отдельной колонке вероятность негативного твита (внутри поля probability представлен вектор из двух значений, нам нужна отдельная колонка с последним значением). Напишите udf который будет доставать эту колонку отдельно. 

In [8]:
val model = PipelineModel.load(modelPath)
val predictionsDF = model.transform(inputDF)

model: org.apache.spark.ml.PipelineModel = pipeline_fa230c4baa13
predictionsDF: org.apache.spark.sql.DataFrame = [tweet: string, words: array<string> ... 4 more fields]


In [11]:
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show(5, false)
}.start()

res6: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@3794bc24


+----------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------+----------------------------------------+----------+
|tweet                                                                                                                             |words                                                                                                                                                    |features                                                                                                                                               

+-------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+----------------------------------------+----------+
|tweet                                                                                                                          |words                                                                                                                                                  |features                                                                                                                                                        |rawPrediction            

+----------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+----------------------------------------+----------+
|tweet                                                                                                                                   |words                                                                                                                                                          |features                                                                                                    

+------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+----------------------------------------+----------+
|tweet                                                                                                             |words                                                                                                                                    |features                                                                                                                                                                                 |rawPrediction                

...

+--------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+---------------------------------------+----------+
|tweet                                                                                                                     |words                                                                                                                                          |features                                                                                                                                                                  |rawPrediction                   

+----------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------+----------------------------------------+----------+
|tweet                                                                                                                       |words                                                                                                                                               |features                                                                                                                                                             

+-------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+----------------------------------------+----------+
|tweet                                                                                                              |words                                                                                                                                |features                                                                                                                                        |rawPrediction                        |probability                             |prediction|
+-------------------------


+-------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+----------------------------------------+----------+
|tweet                                                                                                                    |words                                                                                                                                          |features                                                                                                                                                                 |rawPrediction                     


+----------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------+----------------------------------------+----------+
|tweet                                                                                                                 |words                                                                                                                                    |features                                                                                                                                                  |rawPrediction                          |probability                             

In [14]:
import spark.implicits._
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF
        .withColumn("clean_probability", getProbability($"probability"))
        .show
}.start()

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|sitting around, c...|[sitting, around,...|(1000,[273,372,42...|[5.09002596124897...|[0.50900259612489...|       0.0| 0.4909974038751022|
|@psalmist_one Oka...|[@psalmist_one, o...|(1000,[17,44,99,2...|[5.18401936953344...|[0.51840193695334...|       0.0| 0.4815980630466554|
|YES! CAM WON!  DA...|[yes!, cam, won!,...|(1000,[52,256,295...|[5.59906724119459...|[0.55990672411945...|       0.0| 0.4400932758805407|
|oh for goodness' ...|[oh, for, goodnes...|(1000,[38,73,203,...|[5.09002596124897...|[0.50900259612489...|       0.0| 0.4909974038751022|
|im gonna miss thi...|[im, gonna, 

import spark.implicits._
getProbability: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4494/0x000000084149a840@3b2ff7b8,DoubleType,List(Some(class[value[0]: vector])),None,false,true)
res9: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@79a05acb


+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|@louise__  So you...|[@louise__, , so,...|(1000,[11,17,37,3...|[6.00752300760937...|[0.60075230076093...|       0.0| 0.3992476992390628|
|@erynsays omg hea...|[@erynsays, omg, ...|(1000,[69,162,178...|[5.30662691812699...|[0.53066269181269...|       0.0|0.46933730818730035|
|Man thats one sti...|[man, thats, one,...|(1000,[135,194,25...|[4.86096528196380...|[0.48609652819638...|       1.0| 0.5139034718036193|
|@MeeMeeMatchstic ...|[@meemeematchstic...|(1000,[7,150,220,...|[5.21543350259525...|[0.52154335025952...|       0.0|0.47845664974047447|
|@tirsen Yes, Beer...|[@tirsen, ye

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|I think i have a ...|[i, think, i, hav...|(1000,[169,299,33...|[5.23992833210665...|[0.52399283321066...|       0.0| 0.4760071667893343|
|Didn't get the jo...|[didn't, get, the...|(1000,[17,112,209...|[5.05574026095775...|[0.50557402609577...|       0.0|0.49442597390422405|
|my sis leaves tom...|[my, sis, leaves,...|(1000,[2,25,310,3...|[5.82754113049202...|[0.58275411304920...|       0.0|0.41724588695079723|
|I've got an upset...|[i've, got, an, u...|(1000,[10,310,344...|[5.02952448837956...|[0.50295244883795...|       0.0|0.49704755116204347|
|@TankaBar_Linda U...|[@tankabar_l

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|is sad that @darl...|[is, sad, that, @...|(1000,[148,209,40...|[5.80286646240623...|[0.58028664624062...|       0.0| 0.4197133537593764|
|@staceeadams That...|[@staceeadams, th...|(1000,[38,41,86,1...|[5.28896667478232...|[0.52889666747823...|       0.0| 0.4711033325217676|
|is hopin her car ...|[is, hopin, her, ...|(1000,[17,112,118...|[4.96371594077088...|[0.49637159407708...|       1.0| 0.5036284059229115|
|study  saaave me,...|[study, , saaave,...|(1000,[17,224,372...|[4.86096528196380...|[0.48609652819638...|       1.0| 0.5139034718036193|
|@mattbulian im so...|[@mattbulian

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|Alive &amp; Kicki...|[alive, &amp;, ki...|(1000,[87,99,209,...|[5.19676997473920...|[0.51967699747392...|       0.0|0.48032300252607935|
|Waking up really ...|[waking, up, real...|(1000,[79,112,162...|[5.23128624492578...|[0.52312862449257...|       0.0| 0.4768713755074213|
|Watching dawsons ...|[watching, dawson...|(1000,[26,275,376...|[5.14812629263089...|[0.51481262926308...|       0.0| 0.4851873707369103|
|@jordanknight can...|[@jordanknight, c...|(1000,[17,381,580...|[4.86096528196380...|[0.48609652819638...|       1.0| 0.5139034718036193|
|@die_lavish haha!...|[@die_lavish

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|dirty sexy money ...|[dirty, sexy, mon...|(1000,[37,103,168...|[4.83200563511887...|[0.48320056351188...|       1.0| 0.5167994364881127|
|we are having a M...|[we, are, having,...|(1000,[17,58,120,...|[5.18236947640275...|[0.51823694764027...|       0.0| 0.4817630523597244|
|Going to starbuck...|[going, to, starb...|(1000,[38,69,122,...|[5.02952448837956...|[0.50295244883795...|       0.0|0.49704755116204347|
|is sitting in the...|[is, sitting, in,...|(1000,[17,108,169...|[4.98311746110877...|[0.49831174611087...|       1.0| 0.5016882538891225|
|Good morning folk...|[good, morni

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|               tweet|               words|            features|       rawPrediction|         probability|prediction|  clean_probability|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-------------------+
|I woke up and got...|[i, woke, up, and...|(1000,[10,115,120...|[4.62394770060519...|[0.46239477006051...|       1.0| 0.5376052299394808|
|@MaggieDammit I'm...|[@maggiedammit, i...|(1000,[166,263,34...|[4.94254122195096...|[0.49425412219509...|       1.0| 0.5057458778049038|
|The storm knocked...|[the, storm, knoc...|(1000,[17,115,135...|[5.10488687451327...|[0.51048868745132...|       0.0| 0.4895113125486731|
|@beccaalmond you'...|[@beccaalmond, yo...|(1000,[0,344,405,...|[5.42058287338393...|[0.54205828733839...|       0.0| 0.4579417126616068|
|@uniquelyLANA wel...|[@uniquelyla